In [1]:
# P12 - In-class Linear Regress Example
# 스파크의 MLlib을 잘 활용하려면 데이터의 양, feature 추출, 머신러닝 알고리즘 구현, 평가 이 4가지 단계의 파이프 라인이 잘 구성되어야 한다고 느꼈습니다. 그 중 원인 요소들 (feature)을 잘 찾아 결과에 적용될 수 있도록 함수에 공급할 특성(feature)을 찾는 과정 즉, 특성추출이 가장 중요한 단계 같다고 느꼈습니다. 

In [2]:
import findspark

In [3]:
findspark.init('C:\spark-2.4.5-bin-hadoop2.7')

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [6]:
df = spark.read.csv('cruise_ship_info.csv',inferSchema=True, header=True)

In [7]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [9]:
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [10]:
df.groupBy('cruise_line').count().show()

+-----------------+-----+
|      cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [11]:
from pyspark.ml.feature import StringIndexer

In [12]:
indexer = StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')

In [13]:
indexed = indexer.fit(df).transform(df)

In [14]:
indexed.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)]

In [15]:
from pyspark.ml.linalg import Vectors

In [16]:
from pyspark.ml.feature import VectorAssembler

In [17]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [18]:
assembler = VectorAssembler(inputCols = ['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat'],outputCol='features')

In [19]:
output = assembler.transform(indexed)

In [20]:
output.select('features','crew').show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [21]:
output.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0| 

In [22]:
final_data = output.select('features','crew')

In [23]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [24]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               120|
|   mean| 7.992583333333345|
| stddev|3.5057114253799067|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [25]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                38|
|   mean| 7.167631578947368|
| stddev|3.4677318969675386|
|    min|              0.88|
|    max|              13.6|
+-------+------------------+



In [26]:
from pyspark.ml.regression import LinearRegression

In [27]:
lr = LinearRegression(labelCol='crew')

In [28]:
lrModel = lr.fit(train_data)

In [29]:
lrModel.coefficients

DenseVector([-0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 1.0, 0.0])

In [30]:
lrModel.intercept

-2.723573695258284e-15

In [31]:
test_result = lrModel.evaluate(test_data)

In [32]:
test_result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-3.55271367880050...|
|1.776356839400250...|
|-3.99680288865056...|
|-3.55271367880050...|
|3.552713678800501...|
|1.776356839400250...|
|1.776356839400250...|
|3.552713678800501...|
|3.552713678800501...|
|5.329070518200751...|
|                 0.0|
|                 0.0|
|1.776356839400250...|
|                 0.0|
|-4.44089209850062...|
|                 0.0|
|-4.44089209850062...|
|1.776356839400250...|
|1.332267629550187...|
|1.332267629550187...|
+--------------------+
only showing top 20 rows



In [33]:
test_result.rootMeanSquaredError

2.7154812441804097e-15

In [34]:
test_result.meanSquaredError

7.373838387495585e-30

In [35]:
test_result.r2

1.0

In [36]:
unlabeled_data = test_data.select('features')

In [37]:
predictions = lrModel.transform(unlabeled_data)

In [38]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,86.0,21.04,9...| 8.000000000000004|
|[5.0,133.5,39.59,...|13.129999999999999|
|[6.0,30.276999999...| 3.550000000000004|
|[6.0,90.0,20.0,9....| 9.000000000000004|
|[6.0,113.0,37.82,...|11.999999999999996|
|[6.0,158.0,43.7,1...|13.599999999999998|
|[7.0,158.0,43.7,1...|13.599999999999998|
|[8.0,77.499,19.5,...| 8.999999999999996|
|[8.0,91.0,22.44,9...|10.999999999999996|
|[9.0,88.5,21.24,9...|10.299999999999995|
|[10.0,90.09,25.01...|              8.58|
|[10.0,110.0,29.74...|              11.6|
|[11.0,86.0,21.24,...| 9.299999999999999|
|[11.0,90.09,25.01...|              8.48|
|[12.0,25.0,3.88,5...|2.8700000000000006|
|[13.0,30.27699999...|               4.0|
|[13.0,63.0,14.4,7...| 5.310000000000004|
|[13.0,101.509,27....|11.499999999999998|
|[14.0,30.27699999...|3.7299999999999986|
|[14.0,33.0,4.9,5....| 3.239999999999999|
+--------------------+------------